In [1]:
import os
import pandas as pd
import numpy as np
from glob import glob
import nibabel as nib
from nilearn.plotting import plot_img, plot_stat_map, view_img, plot_prob_atlas
from nilearn.regions import connected_label_regions
from nilearn.glm.first_level.hemodynamic_models import spm_hrf
from nilearn.image import concat_imgs, mean_img, index_img
from nilearn.glm.first_level import FirstLevelModel
from nilearn.glm import threshold_stats_img
import matplotlib.pyplot as plt
from nilearn.plotting import plot_design_matrix
from nilearn.interfaces.fmriprep import load_confounds_strategy
from nilearn.plotting import plot_roi
from nilearn.maskers import NiftiMapsMasker, NiftiSpheresMasker
from scipy.interpolate import interp1d
import seaborn as sns


# Homemade functions

In [2]:
# Open a datasets directory. 
fd = os.open("/Users/luisalvarez/Documents/Datasets", os.O_RDONLY)

# Use os.fchdir() method to change the current dir/folder.
os.fchdir(fd)

# Safe check- Print current working directory
print("Current working dir : %s" % os.getcwd())

Current working dir : /Users/luisalvarez/Documents/Datasets


## Load func files for Sub-03 run 1. 

In [3]:
# Load files for sub 001  
sub03_run1_func_path = "MovieData_BIDS_preproc/sub-03/func/sub-03_task-movie_run-01_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz"
sub03_run1_mask_path = "MovieData_BIDS_preproc/sub-03/func/sub-03_task-movie_run-01_space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz"
sub03_events_path = 'MovieData_BIDS_raw/sub-03/func/sub-03_task-movie_run-01_events.csv'
sub03_run1_confounds_path = "MovieData_BIDS_preproc/sub-03/func/sub-03_task-movie_run-01_desc-confounds_timeseries.tsv"


In [4]:
# Calculate mean image to use as background image.
fmri_img = concat_imgs(sub03_run1_func_path)
mean_img = mean_img(fmri_img)

func_file = nib.load(sub03_run1_func_path)
func_data = func_file.get_fdata()

#print("Sub-03 run 1 has shape: %s" % str(func_data.shape))

# Calculate relevant parameters for GLM and ROI time-course analysis.
n_vols = func_data.shape[3]
TR = 2
n_timepoints = n_vols*TR

print("Sub-03 run 1 contains %s" % str(n_timepoints) + " time points.")
print("Sub-03 run 1 contains %s" % str(n_vols) + " vols.")

Sub-03 run 1 has shape: (68, 80, 68, 378)
Sub-03 run 1 contains: 378 vols.


# Process behavioral data. 

In [8]:
sub03_events_df = pd.read_csv(sub03_events_path, index_col=0)

sub03_events_df.head(20)

,participant,expName,label,released,trial_ITI,scale_flip,sound_trailer.started,sound_trailer.stopped,exp_WatchQ.started,exp_WatchQ.stopped,exp_Watch.keys,exp_FeelQ.started,exp_FeelQ.stopped,exp_Feel.keys,exp_Arousal.started,exp_Arousal.stopped,exp_Arousal.keys
0,ah231215,fmritrailer_run2,rh6,1.0,6.0,0.0,117.635831,None,147.735315,151.746823,2.0,151.665969,155.742751,3.0,155.694630,None,2.0
1,ah231215,fmritrailer_run2,uh1,0.0,6.0,1.0,165.619540,195.674011805997,195.674012,199.707160,4.0,199.671155,203.766015,3.0,203.715461,None,2.0
2,ah231215,fmritrailer_run2,rc2,1.0,6.0,1.0,213.670337,None,243.700427,247.733517,4.0,247.669639,251.747882,3.0,251.700221,None,4.0
3,ah231215,fmritrailer_run2,rh2,1.0,2.0,0.0,261.700535,291.73323982799775,291.733240,295.767502,3.0,295.728788,299.813911,3.0,299.749913,None,2.0
4,ah231215,fmritrailer_run2,rh1,1.0,2.0,1.0,305.714593,335.7568738730042,335.756874,339.765825,3.0,339.743652,343.783202,3.0,343.783202,None,2.0
5,ah231215,fmritrailer_run2,rc9,1.0,6.0,0.0,349.721753,None,379.755897,383.787692,3.0,383.764349,387.780805,4.0,387.747459,None,3.0
6,ah231215,fmritrailer_run2,uc4,0.0,2.0,0.0,397.749807,None,427.767757,431.827877,3.0,431.780320,435.812748,4.0,435.746831,None,3.0
7,ah231215,fmritrailer_run2,rh9,1.0,2.0,0.0,441.780005,471.7951010569959,471.795101,475.834535,2.0,475.809778,479.841190,2.0,479.808730,None,2.0
8,ah231215,fmritrailer_run2,uh2,0.0,2.0,1.0,485.798951,515.8166230719944,515.796498,519.806642,4.0,519.806642,523.845952,4.0,523.819513,None,3.0
9,ah231215,fmritrailer_run2,rc6,1.0,2.0,1.0,529.784136,559.8125010149961,559.795409,563.812359,2.0,563.812359,567.863882,1.0,567.830626,None,2.0


In [10]:


# Run a for loop for each row in the df. 
for id in range(sub03_events_df.shape[0]):

    trailer_name = sub03_events_df["label"][id]
    trailer_onset = sub03_events_df["sound_trailer.started"][id]

    # Some participants have missing values on the "sound_trailer.stopped" column
    # to get around this, the following for loop checks if the value is missing, if true
    # it takes the value from the trial onset and adds 30 sec. 
    if(sub03_events_df["sound_trailer.stopped"][id] == "None"):
        trailer_offset = trailer_onset + 30 
    else: 
        trailer_offset = sub03_events_df["sound_trailer.stopped"][id]

    """ 
    For the questionnaire scores, as I understood it. If they were not inverted (["scale_flip"] == 0), then 
    the lower the score the stronger the response. If they were inverted (["scale_flip"] == 1), the higher the 
    score the stronger the response.
    """
    # Check if scaled was flipped and put scores on the same scale. 
    # For me, the most intuitive is that the higher the score, the stronger the response. 
    if(sub03_events_df["scale_flip"][id] == 1):
        trailer_watch_score = sub03_events_df["exp_Watch.keys"][id]
        trailer_arousal_score = sub03_events_df["exp_Arousal.keys"][id]
        trailer_feel_score = sub03_events_df["exp_Feel.keys"][id]
    else: 
        # Does not work for 4-4 -> we get 0 scores. 
        trailer_watch_score = abs(sub03_events_df["exp_Watch.keys"][id] - 4)
        trailer_arousal_score = abs(sub03_events_df["exp_Arousal.keys"][id] - 4)
        trailer_feel_score = abs(sub03_events_df["exp_Feel.keys"][id] - 4)





16

In [13]:
abs(4 - 4)

0